# Óscar Gómez Borzdynski
## Práctica 2
### Procesos Estocásticos

In [7]:
import intrf
import markov
import numpy as np
import sys
import math
from random import uniform

In [8]:
%%html
<style>
    h1, h2, h3 {color: #3d6f91;}
    b, li, ul {color: #5D8AA8;}
</style>

## Métodos y clases previos

Utilizamos la clase Markov proporcionada por el profesor y definimos un método auxiliar que devuelve la matriz de transición en función de los valores de p y q.

In [20]:
# Returns the Probability matrix for our graphs
def get_PT(p=0.3, q=0.0):
    return [[  1-p,    p,    0.0,    0.0,    0.0,    0.0], 
      [  0.0,  1-p,  p/2.0,    0.0,   p/2.0,   0.0],
      [1/4.0,  0.0,  1/4.0,   1/4.0,  1/4.0,   0.0],
      [    q,  0.0,    0.0,   0.9-q,    0.1,   0.0],
      [  0.0,  0.0,    0.0,     0.0,  1/2.0, 1/2.0],
      [  0.0,  0.0,    0.0,    1/4.0, 1/2.0, 1/4.0] 
            
      ]
#
# This class implements a Markov chain. Upon creation, a transition
# matrix and an initial state are given. The chain may be executed and
# it keeps track of the probability that at time t it is in state m.
#
class Markov:

    #
    # Initialization function
    #
    def __init__(self, PT, m_ini):
        self.PT = PT
        self.nstate = len(PT[0])
        self.state = m_ini
        self.steps = 1      # Number of steps executed so far (for the 
                            # computation of averages)
        self.count = self.nstate*[0]  # count of the times a state was visited
        self.count[self.state] = 1

    #
    # Given a state and the transition probability matrix PT, determine a
    # new state with the probabilities prescribed by the matrix. Returns
    # the value of the new state.
    #
    def _transition(self, m):
        p = self.PT[m]
        u = uniform(0.0, 1.0)
        a = 0
        for k in range(len(p)):
            a = a + p[k]
            if u < a:
                return k
        return len(p)-1

    #
    # Makes a step of the Markov chain: generates a new state, updates
    # the current state, the step counter and the state counter. 
    #
    # Returns the new state of the chain
    #
    def step(self):
        mnew = self._transition(self.state)
        self.state = mnew
        self.steps = self.steps+1
        self.count[self.state] = self.count[self.state] + 1
        return mnew

# Ejercicio 1

<b>Simular el funcionamiento de la cadena y hacer una estimacion de conjunto de $h^2_0$ y $h^5_0$ para $q = 0.1$ y $q= 0$.</b>

Primero debemos definir una función que nos devuelva todos los $h$ en función del estado de entrada.

In [21]:
#
# Runs a set of NC chains (with transition PT and initial state m_ini)
# for a total of T steps and at the end returns probability of ever reaching each state
#
def set_run_h(NC, PT, m_ini, T):
    chains = [Markov(PT, m_ini) for _ in range(NC)]
    state_no = len(PT[0])
    state = np.zeros((NC, state_no))
    for k in range(NC):
        state[k][m_ini] = 1
    for t in range(T):
        for k in range(NC):
            state[k][chains[k].step()] = 1

    return state.mean(axis=0)

In [22]:
starting_at = 0
q = 0.0
print(f"q = {q}, comenzando en el estado {starting_at}:")
h = set_run_h(500, get_PT(q=q), starting_at, 1000)
print(f"h2 = {h[2]}, h5 = {h[5]}")
q = 0.1
print(f"q = {q}, comenzando en el estado {starting_at}:")
h = set_run_h(500, get_PT(q=q), starting_at, 1000)
print(f"h2 = {h[2]}, h5 = {h[5]}")

q = 0.0, comenzando en el estado 0:
h2 = 0.54, h5 = 1.0
q = 0.1, comenzando en el estado 0:
h2 = 1.0, h5 = 1.0


Estos valores nos indican que con $q=0$, tenemos una clase absorbente que impide que la cadena pase por el estado $2$ en algunas de las cadenas de la simulación. Además podemos ver que el estado $5$ si pertenece a esa clase absorbente dado que la probabilidad de pasar por él es $1$.

Con $q=1$ la clase anterior desaparece y por ello la cadena es capaz de volver al estado $2$, haciendo que su $h$ sea $1$.

# Ejercicio 2

<b>Simular el funcionamiento de la cadena y hacer una estimacion de conjunto de $k^2_0$ y $k^2_4$ para $q = 0.1$ y $q= 0$.</b>

Primero debemos definir una función que nos devuelva todos los $k$ en función del estado de entrada.

In [23]:
# Runs a set of NC chains (with transition PT and initial state m_ini)
# for a total of T steps and at the end returns the mean time to reach each state
#
def set_run_k(NC, PT, m_ini, T):
    chains = [Markov(PT, m_ini) for _ in range(NC)]
    state_no = len(PT[0])
    state = np.ones((NC, state_no)) * np.inf
    for k in range(NC):
        state[k][m_ini] = 0
    for t in range(T):
        for k in range(NC):
            c = chains[k].step()
            state[k][c] = t if state[k][c] == np.inf else state[k][c]

    return state.mean(axis=0)

In [24]:
starting_at = 0
q = 0.0
print(f"q = {q}, comenzando en el estado {starting_at}:")
k = set_run_k(500, get_PT(q=q), starting_at, 1000)
print(f"k2 = {k[2]}")
q = 0.1
print(f"q = {q}, comenzando en el estado {starting_at}:")
k = set_run_k(500, get_PT(q=q), starting_at, 1000)
print(f"k2 = {k[2]}")

starting_at = 4
q = 0.0
print(f"q = {q}, comenzando en el estado {starting_at}:")
k = set_run_k(500, get_PT(q=q), starting_at, 1000)
print(f"k2 = {k[2]}")
q = 0.1
print(f"q = {q}, comenzando en el estado {starting_at}:")
k = set_run_k(500, get_PT(q=q), starting_at, 1000)
print(f"k2 = {k[2]}")

q = 0.0, comenzando en el estado 0:
k2 = inf
q = 0.1, comenzando en el estado 0:
k2 = 42.726
q = 0.0, comenzando en el estado 4:
k2 = inf
q = 0.1, comenzando en el estado 4:
k2 = 73.068


Podemos ver que obtenemos valores $k$ infinitos, esto se debe a que la media de un conjunto cuando uno de los valores es infinito es infinito. Cuando una cadena no pasa por el estado $n$, el tiempo necesario para llegar a ese estado es inifinito. Dado que, como hemos visto en el ejercicio anterior, la probabilidad de llegar al estado $2$ no es $1$ para $q=0$, es posible que una cadena del conjunto no llegue a pasar por él y por tanto se le asigne un tiempo inifinito.

# Ejercicio 3

<b>Usar el sistema de ecuaciones lineares oportuno para determinar los valores teoricos correspondientes a las cantidades estimadas y comparar con los valores determinados por medio de la simulacion (cuidado: si una cantidad $k$ es $\infty$ la simulacion claramente no puede dar su valor real...  discutir este caso).</b>